In [14]:
import os
import numpy as np
import pandas as pd

# 현재 폴더 경로
current_folder = os.getcwd()
new_folder = os.path.join(current_folder, "processed_data")
os.makedirs(new_folder, exist_ok=True)

# fittings_filled.csv 파일 불러오기
t0_data = pd.read_csv("fittings_filled.csv")

# .dat 파일 목록 가져오기
data_files = [f for f in os.listdir(current_folder) if f.endswith(".dat")]

for file in data_files:
    # 파일 이름에서 V_app 추출
    V_app = float(file.split("V-")[0])
    V_eff = V_app - 1.1
    
    # t0 찾기
    t0 = t0_data.loc[t0_data["V"] == V_app, "t0"].values
    if len(t0) == 0:
        print(f"Warning: t0 not found for V_app = {V_app}V")
        continue
    t0 = t0[0]
    
    # 데이터 로드 (헤더 없음, 두 번째 컬럼이 t)
    data = pd.read_csv(file, header=None, names=["L-z", "t"], delimiter=",")
    
    # 새로운 컬럼 계산
    data["(L-z)^2"] = (data["L-z"])**2
    data["V_eff*(t-t0)"] = V_eff * (data["t"] - t0)
    data["V_eff*(t)"] = V_eff * (data["t"])
    data["V_eff*(t+t0)"] = V_eff * (data["t"]+t0)
    
    # 새로운 파일 저장
    new_filename = os.path.join(new_folder, f"{V_app}_time_vs_400-z.csv")
    data[["(L-z)^2", "V_eff*(t-t0)", "V_eff*(t)", "V_eff*(t+t0)"]].to_csv(new_filename, index=False, header=False)
    
    print(f"Processed and saved: {new_filename}")

Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\1.8_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\10.0_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\10.2_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\10.5_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\11.0_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\11.5_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data\12.5_time_vs_400-z.csv
Processed and 

In [15]:
import os
import numpy as np
import pandas as pd
from scipy.stats import linregress

# 현재 폴더 경로
current_folder = os.getcwd()
new_folder = os.path.join(current_folder, "processed_data2")
os.makedirs(new_folder, exist_ok=True)

# fittings_filled.csv 파일 불러오기
t0_data = pd.read_csv("fittings_filled.csv")

# .dat 파일 목록 가져오기
data_files = [f for f in os.listdir(current_folder) if f.endswith(".dat")]

for file in data_files:
    # 파일 이름에서 V_app 추출
    V_app = float(file.split("V-")[0])
    V_eff = V_app - 1.1
    
    # t0 찾기
    t0 = t0_data.loc[t0_data["V"] == V_app, "t0"].values
    if len(t0) == 0:
        print(f"Warning: t0 not found for V_app = {V_app}V")
        continue
    t0 = t0[0]
    
    # 데이터 로드 (헤더 없음, 두 번째 컬럼이 t)
    data = pd.read_csv(file, header=None, names=["L-z", "t"], delimiter=",")
    
    # 새로운 컬럼 계산
    data["(L-z)^2"] = (data["L-z"])**2
    data["V_eff*(t-t0)"] = V_eff * (data["t"] - t0)
    data["V_eff*(t)"] = V_eff * (data["t"])
    data["V_eff*(t+t0)"] = V_eff * (data["t"] + t0)
    
    # L-z 범위 필터링 (50~300)
    filtered_data = data[(data["L-z"] >= 50) & (data["L-z"] <= 300)]
    
    # 선형 피팅 수행 및 예측 값 계산
    def linear_fit(x, y, x_full):
        slope, intercept, _, _, _ = linregress(x, y)
        return slope * x_full + intercept
    
    if not filtered_data.empty:
        data["Linear_fit_(L-z)^2_V_eff*(t-t0)"] = linear_fit(filtered_data["V_eff*(t-t0)"], filtered_data["(L-z)^2"], data["V_eff*(t-t0)"])
        data["Linear_fit_(L-z)^2_V_eff*(t)"] = linear_fit(filtered_data["V_eff*(t)"], filtered_data["(L-z)^2"], data["V_eff*(t)"])
        data["Linear_fit_(L-z)^2_V_eff*(t+t0)"] = linear_fit(filtered_data["V_eff*(t+t0)"], filtered_data["(L-z)^2"], data["V_eff*(t+t0)"])
    else:
        data["Linear_fit_(L-z)^2_V_eff*(t-t0)"] = np.nan
        data["Linear_fit_(L-z)^2_V_eff*(t)"] = np.nan
        data["Linear_fit_(L-z)^2_V_eff*(t+t0)"] = np.nan
    
    # 새로운 파일 저장
    new_filename = os.path.join(new_folder, f"{V_app}_time_vs_400-z.csv")
    data.to_csv(new_filename, index=False, header=True)
    
    print(f"Processed and saved: {new_filename}")


Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\1.8_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\10.0_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\10.2_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\10.5_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\11.0_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\11.5_time_vs_400-z.csv
Processed and saved: D:\Forming\Electroforming\RECENT\A3054_Boundary_lines_confirmed_resized_IMPORTANT\processed_data2\12.5_time_vs_400-z.csv
Process